### Load libraries 




In [1]:
import pandas as pd 
import pickle
from IPython import display
import datetime

import os 
import sys

In [2]:
pd.__version__

'1.1.4'

In [3]:
#!pwd
print(os.getcwd())

D:\Documents\Education\University\4th Year\ELEC6200 - Group Design Project\gdp-wild-dogs


### Checking OS system 
- slight difference in file path for different OS systems:
    - linux: '/home/user/Data'
    - windows: 'C:\\user\\Data'

In [4]:
os_system = sys.platform
if os_system.startswith('win'):
    os_system = 'windows'
print('OS System: ', os_system)

OS System:  windows


### List Directory Tree

In [5]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

In [6]:
list_files(os.path.join(os.getcwd(), 'Data'))

Data/
    2020-11-05/
        201105_807d3a2a0fe8_xyz.pkl
        cut.pkl
        cutoff.txt
        labels.csv
        raw.pkl
    2020-11-16/
        201116_807d3a2a0fe8_xyz.pkl
        cut.pkl
        cutoff.txt
        labels.csv
        raw.pkl
    2020-11-23/
        201123_807d3a2a0fe8_xyz.pkl
        cut.pkl
        cutoff.txt
        labels.csv
        raw.pkl
    2020-11-26/
        201126_807d3a2a0fe8_xyz.pkl
        raw.pkl
    2020-11-27/
        201127_807d3a2a0fe8_xyz.pkl
        raw.pkl
    2020-11-28/
        201128_807d3a2a0fe8_xyz.pkl
        raw.pkl


 ### Functions to Trim, Load, and Label data

In [7]:
os_system = sys.platform
if os_system.startswith('win'):
    os_system = 'windows'
print('OS System: ', os_system)

if os_system == 'linux':
    filepath = './Data'
    pathstyle = '/'
elif os_system == 'windows':
    filepath = 'Data'
    pathstyle = '\\'

def trimData(folderPath):
        
    cutoffPresent = 0
    
    for filename in os.listdir(folderPath):
        if filename == "cutoff.txt":
            cutoffPresent = 1
            f = open('./'+folderPath+'/cutoff.txt', "r")
            start = f.readline()
            start = datetime.time(int(start.split(",")[0]), int(start.split(",")[1]), int(start.split(",")[2]))
            end = f.readline()
            end = datetime.time(int(end.split(",")[0]), int(end.split(",")[1]), int(end.split(",")[2]))
    for filename in os.listdir(folderPath):
        if cutoffPresent == 1:
            if filename == "raw.pkl":
                df = pd.read_pickle(subdir+pathstyle+ filename)
                df.reset_index(inplace=True)
                df = df.drop(['mag_x', 'mag_y', 'mag_z', 'pressure'], axis=1)
                display.display(df)
                df = df.loc[(df['dt'].dt.time > start) & (df['dt'].dt.time < end)]
                df.to_pickle("./"+folderPath+"/cut.pkl")
                cutoffPresent = 0
    return 0

def loadData(folderPath):
    for filename in os.listdir(folderPath):
        if filename == "cut.pkl":
            data = pd.read_pickle(folderPath+pathstyle+ filename)
            data.reset_index(inplace=True)
            return data

def loadLabels(folderPath):
    for filename in os.listdir(folderPath):
        if filename == "labels.csv":
            labels = pd.read_csv(folderPath+pathstyle+filename)
            labels.reset_index(inplace=True)
            return labels

def labelData(data, labels):
    if('activity' in data.columns):
                    data.drop('activity', 1, inplace=True)

    labelsList = []
    intervalIndex = 0
    intervalBegin = pd.to_datetime(labels['start_time'][intervalIndex], utc=True)
    intervalEnd = pd.to_datetime(labels['end_time'][intervalIndex], utc=True)
    for i in range(0, len(data)):
        time = pd.to_datetime(data['dt'][i])
        while(time > intervalEnd):
            if(intervalIndex + 1 >= len(labels)):
                break
            intervalIndex += 1
            intervalBegin = pd.to_datetime(labels['start_time'][intervalIndex], utc=True)
            intervalEnd = pd.to_datetime(labels['end_time'][intervalIndex], utc=True)
        if(time > intervalEnd or time < intervalBegin):
            labelsList.append("None")
        else:
            labelsList.append(labels["activity"][intervalIndex])
    data["activity"] = labelsList

def loadAndLabel():
        
    labelledDfList = []

    for subdir, dirs, files in os.walk(filepath):
        if subdir != "Data":
            data = loadData(subdir)
            labels = loadLabels(subdir)
            if (data is not None) & (labels is not None):
                print("Labelled Data from {}".format((str(pd.to_datetime(data['dt'][0], utc=True))[0:10])))
                labelData(data, labels)
                labelledDfList.append(data)
                
    return labelledDfList

OS System:  windows


In [8]:
dfList = loadAndLabel()

Labelled Data from 2020-11-05
Labelled Data from 2020-11-16
Labelled Data from 2020-11-23
